# Churn (attrition) de clients Telecom

On veut prédire le départ de clients d'un opérateur telecom à partir de données comme la formule d'abonnement, ou le temps de communication consommé.

On peut trouver le dataset sur :  
https://www.kaggle.com/mnassrib/telecom-churn-datasets 


## Librairies et fonctions utiles

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from IPython.core.display import HTML # permet d'afficher du code html dans jupyter

Fonction pour standardiser les données quantitatives (cont_feat est une liste des colonnes correspondant à des caractéristiques quantitatives) :

In [ ]:
def scale_feat(df,cont_feat) :
    df1=df
    scaler = preprocessing.RobustScaler()
    df1[cont_feat] = scaler.fit_transform(df1[cont_feat])
    return df1

Fonction pour tracer les courbes d'apprentissage sur l'ensemble d'apprentissage et l'ensemble de validation :

In [ ]:
from sklearn.model_selection import learning_curve
def plot_learning_curve(est, X_train, y_train) :
    train_sizes, train_scores, test_scores = learning_curve(estimator=est, X=X_train, y=y_train, train_sizes=np.linspace(0.1, 1.0, 10),
                                                        cv=5,
                                                        n_jobs=-1)
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)
    plt.figure(figsize=(8,10))
    plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='training accuracy')
    plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
    plt.plot(train_sizes, test_mean,color='green', linestyle='--',marker='s', markersize=5,label='validation accuracy')
    plt.fill_between(train_sizes,test_mean + test_std,test_mean - test_std,alpha=0.15, color='green')
    plt.grid(b='on')
    plt.xlabel('Number of training samples')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.ylim([0.6, 1.0])
    plt.show()

Fonction pour tracer la courbe ROC :

In [ ]:
def plot_roc_curve(est,X_test,y_test) :
    probas = est.predict_proba(X_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,probas[:, 1])
    roc_auc = auc(false_positive_rate, true_positive_rate)
    plt.figure(figsize=(8,8))
    plt.title('Receiver Operating Characteristic')
    plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0,1],[0,1],'r--')        # plus mauvaise courbe
    plt.plot([0,0,1],[0,1,1],'g:')     # meilleure courbe
    plt.xlim([-0.05,1.2])
    plt.ylim([-0.05,1.2])
    plt.ylabel('Taux de vrais positifs')
    plt.xlabel('Taux de faux positifs')
    plt.show

## Traitement du dataset

In [ ]:
df = pd.read_csv("../input/telecom-churn-datasets/churn-bigml-80.csv")

In [ ]:
df.head().T

In [ ]:
df.count()

In [ ]:
df.info()

In [ ]:
df.describe()

On mappe les valeurs de la colonne cible en 0/1 :

In [ ]:
df['Churn'] = df['Churn'].map({ False: 0, True: 1 })

In [ ]:
df.columns

In [ ]:
discr_feat = ['State', 'International plan', 'Voice mail plan', 'Customer service calls','Area code', 'Number vmail messages']
cont_feat = list(set(df.columns) - set(discr_feat)-{'Churn'})

On convertit les catégories en étiquettes numériques :

In [ ]:
for col in discr_feat :
    df[col]=df[col].astype('category')
    df[col] = df[col].cat.codes
    df[col]=df[col].astype('int8')
    

In [ ]:
df.info()

In [ ]:
df.head()

On vérifie s'il y a des valeurs indéterminées dans le dataset :

In [ ]:
df.isnull().values.sum()

Les valeurs numériques ont des caractéristiques très différentes :

In [ ]:
df[cont_feat].describe()

On normalise ces valeurs :

In [ ]:
df=scale_feat(df,cont_feat)

In [ ]:
df[cont_feat].describe()

On affiche les distributions des valeurs continues :

In [ ]:
for col in cont_feat :
    plt.figure(figsize=[10,5])
    sns.kdeplot(df[col])

Est-il nécessaire d'appliquer une transformation sur les distributions ?

## Forêts aléatoires

On construit les ensembles d'apprentissage et de test :

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(['Churn'], axis=1)
y = df.Churn
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

On teste les forêts aléatoires :

In [ ]:
from sklearn import ensemble

rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf))

In [ ]:
cm = confusion_matrix(y_test, y_rf)
print(cm)

## Sous-échantillonage

Il y a beaucoup moins de clients qui partent que de clients qui restent (heureusement pour l'opérateur ...) :

In [ ]:
df.Churn.value_counts()

On va garder autant de clients fidèles que de churners dans l'ensemble d'apprentissage (X_train), en tirant aléatoirement ceux qu'on va garder
On dit qu'on "sous-échantillonne la classe majoritaire"

In [ ]:
from imblearn.under_sampling import RandomUnderSampler 

rus = RandomUnderSampler()
X_train, y_train = rus.fit_sample(X_train, y_train)

On vérifie qu'on a bien équilibré l'ensemble d'apprentissage :

In [ ]:
y_train.value_counts()

On applique les forêts aléatoires sur le nouvel ensemble d'apprentissage

In [ ]:
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

print(classification_report(y_test, y_rf))

cm = confusion_matrix(y_test, y_rf)
print(cm)

On a moins de données d'apprentissage, mais les résultats sont plutôt meilleurs ...

## Suréchantillonnage

On va rééquilibrer le dataset en sur-échantillonnant la classe minoritaire :

La méthode SMOTE (Synthetic Minority Oversampling TEchnique) consiste à synthétiser des éléments pour la classe minoritaire, à partir de ceux qui existent déjà. Elle fonctionne en choisissant au hasard un point de la classe minoritaire et en calculant les k-voisins les plus proches pour ce point. Les points synthétiques sont ajoutés entre le point choisi et ses voisins.

<img src="https://raw.githubusercontent.com/rafjaa/machine_learning_fecib/master/src/static/img/smote.png">

On crée donc de "fausses données" (mais "vraisemblables") pour l'apprentissage

In [ ]:
X = df.drop(['Churn'], axis=1)
y = df.Churn
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_train, y_train = smote.fit_sample(X_train, y_train)

On a bien équilibré l'ensemble d'apprentissage (en "ajoutant" des données) :

In [ ]:
y_train.value_counts()

On teste les forêts aléatoires avec les données suréchantillonnées :

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf))

In [ ]:
cm = confusion_matrix(y_test, y_rf)
print(cm)

Que pensez-vous de cette matrice de confusion ?

In [ ]:
plot_learning_curve(rf, X_train, y_train)

In [ ]:
plot_roc_curve(rf,X_test,y_test)

## Extreme Gradient Boosting : XGBoost avec suréchantillonage SMOTE

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
print(xgb.score(X_test,y_test))

In [ ]:
y_xgb = xgb.predict(X_test)

print(classification_report(y_test, y_xgb))

cm = metrics.confusion_matrix(y_test, y_xgb)
print(cm)

In [ ]:
plot_learning_curve(xgb, X_train, y_train)
plot_roc_curve(xgb,X_test,y_test)

In [ ]:
print(classification_report(y_test, y_xgb))

## XGBoost pondéré

On va utiliser une amélioration de la méthode XGBoost, sans suréchantillonage

On reconstitue les jeux de données sans échantillonnage :

In [ ]:
X = df.drop(['Churn'], axis=1)
y = df.Churn
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
df.Churn.value_counts()

On utilise le paramètre *scale_pos_weight* pour donner plus d'impact aux erreurs commises sur la classe minoritaire :

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(scale_pos_weight=2278/388)
# xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_xgb = xgb.predict(X_test)

In [ ]:
print(classification_report(y_test, y_xgb))
cm = confusion_matrix(y_test, y_xgb)
print(cm)

In [ ]:
plot_learning_curve(xgb, X_train, y_train)
plot_roc_curve(xgb,X_test,y_test)

## Exercice : détection de fraude

Prédire les fraudes à la carte bancaire sur le dataset :  
https://www.kaggle.com/mlg-ulb/creditcardfraud

*Ces données contiennent les transactions effectuées par carte de crédit en septembre 2013 par les détenteurs de cartes européennes.
Cet ensemble de données présente les transactions qui ont eu lieu en deux jours, où nous avons 492 fraudes sur 284 807 transactions. L'ensemble de données est très déséquilibré, la classe positive (fraudes) représente 0,172 % de toutes les transactions.*

*Il ne contient que des variables d'entrée numériques qui sont le résultat d'une transformation ACP (analyse en composantes principales - une méthode de réduction de dimension). Malheureusement, pour des raisons de confidentialité, nous ne pouvons pas fournir les caractéristiques originales et plus d'informations sur le contexte des données. Les caractéristiques V1, V2, ... V28 sont les principales composantes obtenues avec l'ACP, les seules caractéristiques qui n'ont pas été transformées avec l'ACP sont "Temps" et "Montant". La caractéristique "Temps" contient les secondes écoulées entre chaque transaction et la première transaction de l'ensemble de données. La caractéristique "Montant" est le montant de la transaction, cette caractéristique peut être utilisée par exemple pour l'apprentissage dépendant des coûts. La caractéristique 
"Class" est la variable de réponse et prend la valeur 1 en cas de fraude et 0 dans le cas contraire.*